In [ ]:
Обробка даних за допомогою PCA

In [ ]:
#Вивантаження отриманих даних зі сховища на сервері Google Cloud для Польщі (для України - аналогічно)

from google.cloud import storage
import json
TotalTable=[]

client = storage.Client()

bucket = client.get_bucket('%your_bucket_name%')

blob = bucket.blob('TotalTable_Final_Poland.json ')

data_json = blob.download_as_string()

TotalTable = json.loads(data_json)
TotalTable

In [ ]:
#Вивантаження даних з CSV-файлу для Польщі (для України - аналогічно)

import csv

TotalTable = []

with open('TotalTable_Final_Poland.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        cleaned_row = []
        for value in row:
            try:
                cleaned_row.append(float(value.strip("[] ")))
            except ValueError:
                cleaned_row.append(value.strip("[] "))
        TotalTable.append(cleaned_row)

print(TotalTable)

In [ ]:
#(за необхідності) Видалення одинарних лапок у None, якщо вони з'явились при імпортуванні з CSV
for i in range(len(TotalTable)):
    for j in range(len(TotalTable[i])):
        if TotalTable[i][j]=='None':
            TotalTable[i][j]=None

TotalTable

In [ ]:
#Підготовка даних до обробки методом PCA, видаляються поля, які не використовуватимуться і записи, які містять None
import numpy as np
npTotalTable=np.array(TotalTable)
mask=np.all(npTotalTable!=None, axis=1)
newTotalTable=npTotalTable[mask, 3:]
newTotalTable

In [ ]:
#Обробка методом PCA; поле з маркером (+1/-1) на цей час відділяється, а після обробки приєднується назад; цей модуль слід запускати для України  
from sklearn.decomposition import PCA

fire_ind=newTotalTable[:,-1]
newTotalTable=newTotalTable[:,:-1]

pca = PCA(n_components=15)
pca.fit(newTotalTable)
Orthog_TotalTable=pca.transform(newTotalTable)
Orthog_TotalTable = np.hstack((Orthog_TotalTable, fire_ind.reshape(-1, 1)))
Orthog_TotalTable

In [ ]:
#Цей модуль аналогічний попередньому, але використовується вже раніше "навчений" базис PCA; слід запускати для Польщі
fire_ind=newTotalTable[:,-1]
newTotalTable=newTotalTable[:,:-1]
Orthog_TotalTable=pca.transform(newTotalTable)
Orthog_TotalTable = np.hstack((Orthog_TotalTable, fire_ind.reshape(-1, 1)))
Orthog_TotalTable

In [ ]:
#Перевод у формат списку (зручніше для деяких операцій)
TotalTable_Orthog=Orthog_TotalTable.tolist()
TotalTable_Orthog

In [5]:
#Функція пошуку умовних ймовірностей пожеж і не пожеж у вказаному діапазоні
def SearchStat(array, col_index, left_border, right_border):
    total=0
    counter=0
    for i in range(len(array)):
        if array[i][col_index] is None:
            continue
        elif array[i][col_index]>=left_border and array[i][col_index]<right_border:
            total+=1
            if array[i][-1]==1:
                counter+=1
    return counter/total

In [ ]:
#Утворення таблиці статистики (межі рівноймовірних діапазонів атрибутів та умовні ймовріності пожеж та не пожеж)
import numpy as np
StatTable=[]
for j in range(len(Orthog_TotalTable[0])-1):
    temp_arr=Orthog_TotalTable[:, j]
    temp_arr=np.sort(temp_arr)
    B0=temp_arr[0]
    B8=temp_arr[-1]
    B4=np.median(temp_arr)
    left_half, right_half = np.split(temp_arr, [len(temp_arr)//2])
    B2=np.median(left_half)
    B6=np.median(right_half)
    first_quarter, second_quarter = np.split(left_half, [len(left_half)//2])
    third_quarter, fourth_quarter = np.split(right_half, [len(right_half)//2])
    B1=np.median(first_quarter)
    B3=np.median(second_quarter)
    B5=np.median(third_quarter)
    B7=np.median(fourth_quarter)
    StatTable.append([B0, B1, B2, B3, B4, B5, B6, B7, B8])
    coincidences=0
    n=len(StatTable[-1])
    for k in range(n-1):
        if StatTable[-1][k]==StatTable[-1][k+1]:
            coincidences+=1
            continue
        else:
            StatTable[-1].extend([SearchStat(Orthog_TotalTable, j, StatTable[-1][k-coincidences], StatTable[-1][k+1])])
            coincidences=0
StatTable

In [ ]:
#Прогнозування пожеж на основі добутку відношень умовних ймовірностей пожеж і не пожеж 
for i in range(len(TotalTable_Orthog)):
    multiple=1
    for j in range(len(TotalTable_Orthog[i])-1):
        if TotalTable_Orthog[i][j] is not None:
            coincidences=0
            for k in range(8):
                if StatTable[j][k]==StatTable[j][k+1]:
                    coincidences+=1
                    continue
                elif TotalTable_Orthog[i][j]>=StatTable[j][k] and TotalTable_Orthog[i][j]<StatTable[j][k+1]:
                    #if TotalTable_Orthog[i][-1]==1:
                    if StatTable[j][k+9-coincidences]!=1:
                        multiple*=StatTable[j][k+9-coincidences]/(1-StatTable[j][k+9-coincidences])
                    else:
                        multiple*=100
                    break
    TotalTable_Orthog[i].extend([multiple])
TotalTable_Orthog

In [ ]:
#Отримання матриці невідповідностей
A11=0
A12=0
A21=0
A22=0
for tt in TotalTable_Orthog:
    if tt[-2]==1:
        if tt[-1]>=1:
            A11+=1
        else:
            A21+=1
    else:
        if tt[-1]<1:
            A22+=1
        else:
            A12+=1
print(A11)
print(A12)
print(A21)
print(A22)

In [ ]:
#Відбір фактичних показників (маркерів) стану пожежі/не пожежі для побудови ROC-кривої
Orthog_TotalTable=np.array(TotalTable_Orthog)
res_true=Orthog_TotalTable[:, -2]
res_true=res_true.tolist()
res_true

In [ ]:
#Відбір прогнозованих показників стану пожежі/не пожежі для побудови ROC-кривої
res_test=Orthog_TotalTable[:, -1]
res_test=res_test.tolist()
res_test

In [ ]:
#Побудова ROC-кривої
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score

fpr, tpr, thresholds = roc_curve(res_true, res_test)

roc_auc = roc_auc_score(res_true, res_test)

plt.figure()
plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc='lower right')
plt.show()